### Goal:

Automate data cleaning for remaining four years of period data

In [12]:
import gzip
import os
import io
import pandas as pd
import numpy as np
from datetime import datetime, timezone, timedelta

Cleaning portion involves
- selecting only UTC and period columns
- renaming those columns
- getting rid of half hour data

In [1]:
#write gzip file to text file
#gz = gzip file path name, a string
#txt = text file path name, a string

def unzipPeriod(gz, txt):

    #unzipping
    file = gz
    with gzip.open(file, 'rb') as ip:
            with io.TextIOWrapper(ip, encoding='utf-8') as decoder:
                # Let's read the content using read()
                content = decoder.read()
    with open(txt, 'w') as f:
        f.write(content)
    
    return

In [9]:
#cleaning tide data and creating csv
#txt = text file path name, a string
#csv = csv file path name, a string

def cleanPeriod(txt, csv):

    #reading as space delimited file
    df = pd.read_csv(txt, sep='\s+', header=None)
    
    #cleaning
    df=df.iloc[:,[0,2]].rename({0: 'UTC', 2: 'Period'}, axis='columns')

    #getting rid of 30 minute data bc only need hour granularity
    df = df.iloc[::2]
    
    #creating a csv
    df.to_csv(csv, index=False)
    
    return
    

In [7]:
unzipPeriod('data/raw/pcdip2017.gz','data/interim/period2017.txt')
unzipPeriod('data/raw/pcdip2018.gz','data/interim/period2018.txt')
unzipPeriod('data/raw/pcdip2019.gz','data/interim/period2019.txt')
unzipPeriod('data/raw/pcdip2020.gz','data/interim/period2020.txt')
unzipPeriod('data/raw/pcdip2021.gz','data/interim/period2021.txt')

### Had to manually delete top three lines of each txt file to avoid a ```read_csv``` parsing error

In [10]:
cleanPeriod('data/interim/period2017.txt','data/interim/00-period2017.csv')
cleanPeriod('data/interim/period2018.txt','data/interim/00-period2018.csv')
cleanPeriod('data/interim/period2019.txt','data/interim/00-period2019.csv')
cleanPeriod('data/interim/period2020.txt','data/interim/00-period2020.csv')
cleanPeriod('data/interim/period2021.txt','data/interim/00-period2021.csv')

Success!

## New Goal:
append all years together for one dataset

### csv files cannot store datetime objects, so need to convert the UTC column to a datetime object while reading the file to a dataframe. 

In [13]:
date_parser = pd.to_datetime

p17 = pd.read_csv('data/interim/00-period2017.csv', parse_dates=['UTC'], date_parser=date_parser)
p18 = pd.read_csv('data/interim/00-period2018.csv', parse_dates=['UTC'], date_parser=date_parser)
p19 = pd.read_csv('data/interim/00-period2019.csv', parse_dates=['UTC'], date_parser=date_parser)
p20 = pd.read_csv('data/interim/00-period2020.csv', parse_dates=['UTC'], date_parser=date_parser)
p21 = pd.read_csv('data/interim/00-period2021.csv', parse_dates=['UTC'], date_parser=date_parser)

In [14]:
bigp = pd.concat([p17,p18,p19,p20,p21], ignore_index=True)
bigp.shape

(43767, 2)

definitely missing some hours but should be alright

In [15]:
24*365*5+24

43824

In [16]:
#checking to make sure there arn't duplicate dates

date_count = bigp['UTC'].value_counts().to_list()

ones = np.ones(len(date_count))

truth = date_count==ones
truth.sum()==bigp.shape[0]

True

In [ ]:
bigp.to_csv('data/interim/00-period.csv',index=False)

## Half Hour to Top of Hour
Ran into some merging trouble, turns out it was from almost all of 2021 data being taken at the half hour instead of the top of the hour. 

In [17]:
def checkHour(df):
    df['UTC'] = pd.to_datetime(df['UTC'], utc=True)
    onehour = timedelta(days=0, hours=1)
    baddies = []
    index = df.index.to_list()
    index.remove(0)
    for i in index:
        if (df['UTC'][i]-df['UTC'][i-1] != onehour):
            baddies.append(i)
    return len(baddies), baddies

In [18]:
badi = checkHour(bigp)[1]


In [19]:
i=3
bigp.iloc[badi[i]-2:badi[i]+3]

,UTC,Period
35005,2020-12-31 22:30:00+00:00,7
35006,2020-12-31 23:30:00+00:00,4
35007,2021-01-01 00:00:00+00:00,7
35008,2021-01-01 01:00:00+00:00,15
35009,2021-01-01 02:00:00+00:00,15


Missing 7pm 10/4/2018 <br/>
Missing all of 6/18/2019 and the first 18 hours of 6/19 and the last 12 hours of 6/17 <br/>
**On 2/3/2020 at 2:30pm measurements are taken at the half hour instead of the hour** <br/>
**In the first hour of 2021, measurements go back to top of the hour** <br/>

SOooooo all those missing values in the merge are becasue almost a year's worth of period data is at the half hour, not the hour, so gotta change those measurements....not gonna worry about how the accuracy of data changes because only a 30min difference, not a big deal 

In [20]:
bigp['UTC'].iloc[27029:35007] = bigp['UTC'].iloc[27029:35007]-timedelta(minutes=30)

C:\Users\audre\AppData\Local\Temp\ipykernel_392\678768183.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bigp['UTC'].iloc[27029:35007] = bigp['UTC'].iloc[27029:35007]-timedelta(minutes=30)


In [23]:
checkHour(bigp)

(2, [15403, 21538])

In [21]:
bigp.to_csv('data/interim/01-period.csv',index=False)

In [22]:
bigp.isna().sum()

UTC       0
Period    0
dtype: int64